In [30]:
import os
from typing import List


In [31]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [32]:
from text_summarizer import *
from text_summarizer.utils.common import read_yaml,create_directories


In [33]:
# Use absolute paths for configuration files
CONFIG_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\config\\config.yaml"
PARAMS_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\params.yaml"

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        # Convert the file paths to pathlib.Path objects
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        # Print the paths when initializing
        print("Config File Path:", config_filepath)
        print("Params File Path:", params_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        # Convert root_dir to Path object
        root_dir = Path(config.root_dir)

        # Print for debugging
        print("root_dir (after conversion):", root_dir)

        create_directories([root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=root_dir,
            data_path=Path(config.data_path),
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config


In [34]:
import os
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [35]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))    

In [37]:
# Main execution
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e


Config File Path: D:\projectFiles\text_summarization_project\config\config.yaml
Params File Path: D:\projectFiles\text_summarization_project\params.yaml
[2023-12-30 17:25:34,995: INFO: common:yaml file: D:\projectFiles\text_summarization_project\config\config.yaml loaded successfully]
[2023-12-30 17:25:35,013: INFO: common:yaml file: D:\projectFiles\text_summarization_project\params.yaml loaded successfully]
[2023-12-30 17:25:35,018: INFO: common:created directory at: artifacts]
Config (before conversion): root_dir:artifacts/data_transformation data_path:artifacts/data_ingestion/samsum_dataset tokenizer_name:google/pegasus-cnn_dailymail


AttributeError: 'str' object has no attribute 'root_dir'

In [38]:
# Main execution
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e


Config File Path: D:\projectFiles\text_summarization_project\config\config.yaml
Params File Path: D:\projectFiles\text_summarization_project\params.yaml
[2023-12-30 17:26:12,401: INFO: common:yaml file: D:\projectFiles\text_summarization_project\config\config.yaml loaded successfully]
[2023-12-30 17:26:12,413: INFO: common:yaml file: D:\projectFiles\text_summarization_project\params.yaml loaded successfully]
[2023-12-30 17:26:12,420: INFO: common:created directory at: artifacts]
Config (before conversion): root_dir:artifacts/data_transformation data_path:artifacts/data_ingestion/samsum_dataset tokenizer_name:google/pegasus-cnn_dailymail


AttributeError: 'str' object has no attribute 'root_dir'